In [7]:
import math as ma
import random as ra
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time as t
import ast

In [9]:
def getpopnameDriver(pap, driverPrice):
    def getDrivername(individu, driverPrice):
        cost = fnObj(individu, driverPrice)
        cost = pd.DataFrame(cost)
        cost['cluster'] = [str(a) for a in cost['cluster']]
        result = pd.merge(individu, cost[['cluster', 'driver']], how = 'left', on='cluster')
        return result
    popDriver = []
    for ind in pap:
        driverPop = getDrivername(ind, driverPrice)
        popDriver.append(driverPop)
    return popDriver

In [10]:
"""main program GA DEF"""
def gaMS(dfClient, dismat, capmaxDem, capmaxBox, capminDem, popSize, loclien, driverPrice, numParents, numClient, mutRate, numChild, ranRate):
    timeStart = t.time()
    y=[]
    xTime = [timeStart-t.time()]
    xGen = [1]
    gen = []
    imp = []
    matingPool = {}
    cont = True
    pop = {'pop':[], 'fitness':[]}
    timeGen1 = t.time()
    pop['pop'] = getranpopMS(dfClient, dismat, capmaxDem, capmaxBox, capminDem, popSize, loclien, ranRate)
    pop['fitness'] = getpopFitness(pop['pop'], driverPrice)
    timeGen2 = t.time()
    bestCost = max(pop['fitness'])
    initCost = bestCost.copy()
    print("Biaya pengiriman: ", 1/bestCost)
    popAwal = pop.copy()
    y.append(1/bestCost)
    found = []
    while cont:
        children = {}
        matingPool = getParents(pop, numParents)
        children = getChildren(pop, matingPool, numClient, mutRate, 
                               numChild, driverPrice, capmaxBox, capminDem, capmaxDem, loclien)
        if len(children['pop']) > 0:
            pop = newgenMiu(pop, children, popSize)
        y.append(1/max(pop['fitness']))
        xTime.append(t.time()-timeStart)
        gen.append(1)
        xGen.append(len(gen))
        nowCost = max(pop['fitness'])
        if nowCost > bestCost:
            bestCost = nowCost
            imp = []
            print('Improved')
            print('Biaya pengiriman: ', 1/bestCost)
        else:
            imp.append(1)
        if len(gen) == maxGen:
            cont = False
        if len(imp) == maxIter:
            cont = False
    
    pa = pd.DataFrame(pop)
    best = pa[pa['fitness'] == max(pa['fitness'])]
    best = best.reset_index(drop = True)
    temp = best.loc[0]
    best = temp.copy()
    result = {
            'initialCost': [1/initCost], 
            'finalCost': [1/max(pop['fitness'])],
            'runTimeSW' : [timeGen2 - timeGen1],
            'runTimeGA': [t.time()-timeStart],
            'totdist':[sum(best['pop']['dist'])],
            'numDriver':[len(best['pop']['cluster'].unique())]
    }
    popAwal['pop'] = getpopnameDriver(popAwal['pop'], driverPrice)
    popAkhir = pop.copy()
    popAkhir['pop'] = getpopnameDriver(popAkhir['pop'], driverPrice)
    
    graph = {'xTime': xTime, 'xGen': xGen, 'y': y}
    result = pd.DataFrame(result)
    return result, popAwal, popAkhir, graph

In [11]:
def generateranDismat(jumklien):
    loklien = [[0,0]]
    LOCLIEN = {'lat':[0], 'lon':[0]}
    dismat = {}
    numKlien = 0
    for i in range(1, jumklien+1):
        a = ra.gauss(1.909871429,4.732033696)
        b = ra.gauss(-1.398453571,7.153205539)
        LOCLIEN['lat'].append(a)
        LOCLIEN['lon'].append(b)
        loklien.append([a, b])
    for i in loklien:
        temp = []
        for j in loklien:
            dist = ma.sqrt((i[0]-j[0])**2 + (i[1]-j[1])**2)
            temp.extend([dist])
        dismat[str(numKlien)] = temp
        numKlien +=1
    return dismat, loklien, LOCLIEN

def bignum(dm):
    data2 = {}
    for a in dm:
        data3 = []
        for b in dm[a]:
            if b == 0:
                b = 1000
            data3.append(b)
        data2[a] = data3
    return data2

def generatedismatTemp(rt, dismat):
    dm = dismat.copy()
    disTemp1 = {}
    disTemp = {}
    bt = [int(i) for i in rt]
    bt.sort()
    sortRT = [str(i) for i in bt]
    for a in sortRT:
        disTemp1[a] = dm[a]
    for a in disTemp1:
        disTemp2 = []
        for b in sortRT:
            disTemp2.append(disTemp1[a][int(b)])
        disTemp[a] = disTemp2
    return disTemp

def getminDriver(driverCostsum, minDist, row):
    dt = {}
    dt['column'] = list(driverCostsum.iloc[row].index)
    dt['value'] = list(driverCostsum.iloc[row])
    idx = dt['value'].index(minDist)
    return dt['column'][idx]

In [12]:
"""Get Degree"""
"""lonCent dan latCent harus = 0"""
def getlDist(df, lonCent, latCent):
    if lonCent != 0 or latCent != 0:
        print("Error. Lokasi depot tidak di titik (0, 0). Hasil tidak valid")
    else:
        jarak_lon = abs(df['lon'] - lonCent)
        jarak_lat = abs(df['lat'] - latCent)
        df['lonDist'] = jarak_lon.copy()
        df['latDist'] = jarak_lat.copy()
    return df

def complement(df, lonCent, latCent):
    sudut = []
    for i in range(len(df)):
        if df['lon'][i] > 0:
            if df['lat'][i] > 0:
                deg = np.arctan(df['lonDist'][i]/df['latDist'][i])/(2*np.pi)*360
                sudut.append(deg)
            elif df['lat'][i] == 0:
                deg = 90
                sudut.append(deg)
            else:
                deg = (np.arctan(df['lonDist'][i]/df['latDist'][i])/(2*np.pi)*360) + 90
                sudut.append(deg)
        elif df['lon'][i] == 0:
            if df['lat'][i] >= 0:
                deg = 0
                sudut.append(deg)
            else:
                deg = 180
                sudut.append(deg)
        else:
            if df['lat'][i] > 0:
                deg = (np.arctan(df['lonDist'][i]/df['latDist'][i])/(2*np.pi)*360) + 270
                sudut.append(deg)
            elif df['lat'][i] == 0:
                deg = 270
                sudut.append(deg)
            else:
                deg = (np.arctan(df['lonDist'][i]/df['latDist'][i])/(2*np.pi)*360) + 180
                sudut.append(deg)
    df['deg'] = sudut.copy()
    df = df.sort_values(by=['deg']).reset_index(drop = True)    
    return df


def getDeg(loca, df):
    ca = {'lat':[], 'lon':[]}
    for a in loca[1:]:
        for b in range(len(a)):
            if b==0:
                ca['lat'].append(a[b])
            else:
                ca['lon'].append(a[b])
    latCent = loca[0][0]
    lonCent = loca[0][1]
    dfClient['lat'] = ca['lat'].copy()
    dfClient['lon'] = ca['lon'].copy()
    df = getlDist(df, lonCent, latCent)
    df = complement(df, lonCent, latCent)
    del df['lonDist'], df['latDist']
    return df

def rotate(dfClient_deg):
    if ra.random() < 0.1:
        dfd = dfClient_deg.sort_values(by=['deg'], ascending = True).reset_index(drop = True)
    else:
        dfd = dfClient_deg.sort_values(by=['deg'], ascending = False).reset_index(drop = True)
    dfd = pd.DataFrame(dfd)
    dfd1 = dfd.loc[0].copy()
    dfd2 = dfd.loc[1:].copy()
    dfd = dfd2.append(dfd1)
    dfd = dfd.reset_index(drop = True)
    return dfd

def rotateRan(df, chosenClient, ccw = True):
    dfClient1 = df.copy()
    if ccw:
        dfClient1 = dfClient1.sort_values(by=['deg'], ascending = True).reset_index(drop = True)
        chosenIndex = list(dfClient1.index[dfClient1['route']==chosenClient])[0]
        chosenIndex = range(chosenIndex)
        notchosenIndex = [a for a in range(len(dfClient1)) if a not in chosenIndex]
        parent1 = dfClient1.reindex(chosenIndex).copy()
        parent2 = dfClient1.reindex(notchosenIndex).copy()
        parent = parent2.append(parent1)
    else:
        dfClient1 = dfClient1.sort_values(by=['deg'], ascending = False).reset_index(drop = True)
        chosenIndex = list(dfClient1.index[dfClient1['route']==chosenClient])[0]
        chosenIndex = range(chosenIndex)
        notchosenIndex = [a for a in range(len(dfClient1)) if a not in chosenIndex]
        parent1 = dfClient1.reindex(chosenIndex).copy()
        parent2 = dfClient1.reindex(notchosenIndex).copy()
        parent = parent2.append(parent1)
    return parent.reset_index(drop=True)

In [13]:
def renumberInd(ba, loca):
    firstClus = []
    latCent = loca[0][0]
    lonCent = loca[0][1]
    di = {'cluster':[], 'lon':[], 'lat':[]}
    clusters = list(ba['cluster'].unique())
    chosen = ba.copy()
    for a in clusters:
        di['cluster'].append(a)
        di['lat'].append(np.mean(chosen['lat'][chosen['cluster']==a]))
        di['lon'].append(np.mean(chosen['lon'][chosen['cluster']==a]))
    di = pd.DataFrame(di)
    di = getlDist(di, lonCent, latCent)
    di = complement(di, lonCent, latCent)
    di = di.reset_index(drop = True)
    for deg in di['deg']:
        if deg >= 180:
            firstClus.append(360-deg)
        else:
            firstClus.append(deg)
    di['degNorm'] = firstClus
    first = di[di['degNorm'] == np.min(di['degNorm'])].copy()
    firstComp = [i for i in list(di.index) if i not in first.index]
    di = di.reindex(firstComp).sort_values(by=['deg'])
    first = first.append(di).reset_index(drop = True)
    newCluster = [str(i) for i in range(len(first))]
    first['newCluster'] = newCluster.copy()
    tempClus = []
    for a in chosen['cluster'].unique():
        newCluster = first['newCluster'][first['cluster'] == a].copy()
        for i in chosen['cluster']:
            if i == a:
                tempClus.append(str(int(newCluster)))
    
    del chosen['cluster'] 
    chosen['cluster'] = tempClus.copy()
    return chosen

def renumberPop(py, loclien):
    pr = [i for i in py]
    ab = []
    for a in range(len(pr)):
        pg = renumberInd(pr[a], loclien)
        ab.append(pg)
    return ab

In [14]:
def getranpopMS(dfClient, dismat, capmaxDem, capmaxBox, capminDem, popSize, loclien, ranRate):
    print("Mencari solusi untuk populasi awal dengan Sweep (konstruksi)")
    mao = []
    dfClient_deg = getDeg(loclien, dfClient)
    for a in range(popSize):
        fs = False
        while fs == False:
            chosenClient = ra.choice(list(dfClient_deg['route']))
            if ra.random() < 0.5:
                neDf = rotateRan(dfClient_deg, chosenClient, True)
                neDf, fs = clusterIt2(neDf, dismat, capmaxDem, capmaxBox, capminDem, ranRate)
            else:
                neDf = rotateRan(dfClient_deg, chosenClient, False)
                neDf, fs = clusterIt2(neDf, dismat, capmaxDem, capmaxBox, capminDem, ranRate)
            neDf = nnInd(neDf, dismat)
            neDf = renumberInd(neDf, loclien)
            neDf = sortclusInd(neDf)
            neDf = neDf.reset_index(drop = True)
            if fs:
                if len(mao) > 0:
                    fs = getfsSame(neDf, mao)
        print("Solusi ", len(mao) + 1, "ditemukan")
        mao.append(neDf)
    print("Memperbaiki solusi dengan algoritma genetika")
    return mao

In [15]:
def sortclusInd(pa):
    ba= pa.copy()
    tempClust = {'cluster':[], 'pop':[]}
    for a in ba['cluster'].unique():
        temp = ba[ba['cluster']==a].copy()
        tempClust['pop'].append(temp)
        tempClust['cluster'].append(int(a))
    tempClust = pd.DataFrame(tempClust)
    tempClust = tempClust.sort_values(by=['cluster']).reset_index(drop =True)
    tempClust['cluster'] = tempClust['cluster'].astype(str)
    first = True
    for a in tempClust['pop']:
        if first:
            tempDf = a.copy()
            first = False
        else:
            tempDf = tempDf.append(a)
    tempDf = tempDf.reset_index(drop = True)
    return tempDf

def sortrouteInd(pa):
    ba = pa.copy()
    ba['route'] = ba['route'].astype(int)
    ba = ba.sort_values(by=['route'])
    ba['route'] = ba['route'].astype(str)
    ba = ba.reset_index(drop = True)
    return ba

In [16]:
def nn(dT1):
    tempRoute = []
    distance = []
    dmt = bignum(dT1)
    dismatTemp = dmt.copy()
    fromCity = '0'
    ba = len(dismatTemp)
    for b in dismatTemp:
        dismatTemp[b][0] = 1000
    batas = len(dismatTemp)-1
    i = 0
    while i < batas:
        dist = min(dismatTemp[fromCity])
        tocityIndex = dismatTemp[fromCity].index(dist)
        dismatTemp[fromCity] = [1000 for i in range(len(dismatTemp[fromCity]))]
        for b in dismatTemp:
            dismatTemp[b][tocityIndex] = 1000
        toCity = list(dismatTemp.keys())[tocityIndex]
        fromCity = toCity
        tempRoute.append(toCity)
        distance.append(dist)
        i = i+1
    return tempRoute, distance

def nnInd(be, dismat):
    first = True
    dY = be.copy()
    for a in be['cluster'].unique():
        tempClus = ['0']
        temp = be['route'][be['cluster'] == a].copy()
        tempClus.extend(list(temp))
        dT = generatedismatTemp(list(tempClus), dismat)
        dT, distance = nn(dT)
        if first:
            pa = dY[dY['cluster']==a]
            pa = pa.set_index(pa['route'])
            pa = pa.reindex(dT)
            pa = pa.reset_index(drop = True)
            first = False
        else:
            da = dY[dY['cluster']==a]
            da = da.set_index(da['route'])
            da = da.reindex(dT)
            da = da.reset_index(drop = True)
            pa = pa.append(da)
    pa = pa.reset_index(drop = True)
    pa = getDist(pa, dismat)
    return pa

def nnPop(py, dismat):
    pr = [i for i in py]
    ab = []
    for a in range(len(pr)):
        pg = nnInd(pr[a], dismat)
        ab.append(pg)
    return ab

In [17]:
def newgenMiu(pop1, children1, popSize):
    pop3 = {'pop':[i for i in pop1['pop']],
            'fitness': [i for i in pop1['fitness']]}
    children3 = {'pop':[i for i in children1['pop']],
                'fitness': [i for i in children1['fitness']]}
    pa = pd.DataFrame(pop3)
    ch = pd.DataFrame(children3)
    pa = pa.append(ch, ignore_index = True)
    pa = pa.sort_values(by=['fitness'], ascending= False).reset_index(drop = True)
    da = pa[0:popSize].copy()
    da = da.to_dict(orient='list')
    pop2 = da.copy()
    return pop2

In [18]:
def getParents(pop1, numParents):
    matingPool1 = {'pop':[], 'fitness':[]}
    for a in range(numParents):
        tempParents = {'fitness':[], 'population':[]}
        tempParent = ra.sample(list(range(len(pop1['pop']))), 2)
        for b in tempParent:
            tempParents['fitness'].append(pop1['fitness'][b])
            tempParents['population'].append(pop1['pop'][b])
        idx = tempParents['fitness'].index(max(tempParents['fitness']))
        tP = tempParents['population'][idx].reset_index(drop = True)
        tQ = tempParents['fitness'][idx]
        matingPool1['pop'].append(tP)
        matingPool1['fitness'].append(tQ)
    return matingPool1

In [27]:
def getChildren(old, matingPool, numClient, mutRate, numChild, driverPrice, capmaxBox, capminDem, capmaxDem, loclien):
    sormatingPool = []
    temp2 = {'pop':[]}
    ba = [i for i in matingPool['pop']]
    children1 = {'pop':[], 'fitness':[]}
    child1 = []
    for a in range(len(matingPool['pop'])):
        ba[a]['route'] = ba[a]['route'].astype(int)
        ba[a] = ba[a].sort_values(by=['route']).reset_index(drop = True)
        ba[a]['route'] = ba[a]['route'].astype(str)
        sormatingPool.append(ba[a])
    startGene = int(ra.random()* numClient)
    endGene = int(ra.random()* numClient)
    while startGene == endGene:
        startGene = int(ra.random()* numClient)
        endGene = int(ra.random()* numClient)
    gene1 = min(startGene, endGene)
    gene2 = max(startGene, endGene)
    pa1, pa2 = ra.sample(list(range(len(sormatingPool))), 2)
    CP1 = sormatingPool[pa1].copy()
    CP2 = sormatingPool[pa2].copy()

    genP1 = sormatingPool[pa1].loc[gene1:gene2].copy()
    genP2 = sormatingPool[pa2].loc[gene1:gene2].copy()

    CP1.loc[gene1:gene2] = genP2.copy()
    CP2.loc[gene1:gene2] = genP1.copy()

    CP1 = CP1.reset_index(drop = True)
    CP2 = CP2.reset_index(drop = True)
    
    CP1 = nnInd(CP1, dismat)
    CP1 = renumberInd(CP1, loclien)
    CP1 = sortclusInd(CP1)
    CP1 = getDist(CP1, dismat)
    
    CP2 = nnInd(CP2, dismat)
    CP2 = renumberInd(CP2, loclien)
    CP2 = sortclusInd(CP2)
    CP2 = getDist(CP2, dismat)
    
    CP1 = CP1.reset_index(drop = True)
    CP2 = CP2.reset_index(drop = True)
    
    fsCP1 = getfsCluster(CP1, capmaxBox, capminDem, capmaxDem)
    fsCP2 = getfsCluster(CP2, capmaxBox, capminDem, capmaxDem)
    
    if fsCP1:
        child1.append(CP1)
    if fsCP2:
        temp2['pop'].append(CP2)
        fsCP2 = getfsSame(CP1, temp2['pop'])
        if fsCP2:
            child1.append(CP2)
    
    if len(child1) > 0:
        for ch in range(len(child1)):
            if ra.random() < mutRate:
                childMutated = mutate(child1, ch)
                childMutated = renumberInd(childMutated[0], loclien)
                childMutated = nnInd(childMutated, dismat)
                childMutated = sortclusInd(childMutated)
                childMutated = childMutated.reset_index(drop = True) 
                fs = getfsCluster(childMutated, capmaxBox, capminDem, capmaxDem)
                if fs:
                    fs = getfsSame(childMutated, child1)
                    if fs:
                        childMutated = getDist(childMutated, dismat)
                        child1[ch] = childMutated.reset_index(drop = True)
        for ch in range(len(child1)):
            fs = True
            fs = getfsSame(child1[ch], old['pop'])
            if fs:
                dQ = getFitness(child1[ch], driverPrice)
                children1['pop'].append(child1[ch])
                children1['fitness'].append(dQ)
    return children1

In [20]:
def mutate(sormatingPool1, child):
    sormatingPool1[child] = sortclusInd(sormatingPool1[child])
    gene1, gene2 = ra.sample(list(sormatingPool1[child].index), 2)
    while sormatingPool1[child]['cluster'][gene1] == sormatingPool1[child]['cluster'][gene2]:
        gene1, gene2 = ra.sample(list(sormatingPool1[child].index), 2)
    geneData1 = sormatingPool1[child].loc[gene1].copy()
    geneData2 = sormatingPool1[child].loc[gene2].copy()
    sormatingPool1[child].loc[gene1] = geneData2.copy()
    sormatingPool1[child].loc[gene2] = geneData1.copy()
    sormatingPool1[child] = sormatingPool1[child].reset_index(drop = True)
    return sormatingPool1

In [21]:
def getFitness(dfsolTemp, driverPrice):
    cost = fnObj(dfsolTemp, driverPrice)
    return 1/sum(cost['cost'])

def getpopFitness(pop1, driverPrice):
    popFitness = []
    for individu in pop1:
        fitPop = getFitness(individu, driverPrice)
        popFitness.append(fitPop)
    return popFitness

def getfsCluster(db, capmaxBox, capminDem, capmaxDem):
    for a in db['cluster'].unique():
        ba = db[db['cluster']==a]
        sumBox = sum(ba['box'])
        sumDem = sum(ba['demand'])
        if sumBox <= capmaxBox and sumDem <= capmaxDem and sumDem >= capminDem:
            fsClus = True
        else:
            fsClus = False
            break
    return fsClus

def getfsSame(child1, pop1):
    sameList = []
    for a in range(len(pop1)):
        same = True
        for b in range(len(child1['route'])):
            if child1['route'][b] != pop1[a]['route'][b]:
                same = False
                sameList.append(same)
                break
        if same:
            sameList.append(same)
            break
    for a in sameList:
        if a:
            popSame = False
            break
        else:
            popSame = True
    return popSame

In [22]:
"""membuat fungsi menghitung biaya"""
def getminDriver(driverCostsum, minDist, row):
    dt = {}
    dt['column'] = list(driverCostsum.iloc[row].index)
    dt['value'] = list(driverCostsum.iloc[row])
    idx = dt['value'].index(minDist)
    return dt['column'][idx]

def getdriverCostsum(dP, dtS):
    driverCostsum = {i:[] for i in dP['driver']}
    for a in range(0, len(dtS['cluster'])):
        for dr in dP['driver']:
            costTemp = getdriverCost(dP, dtS, a)
            driverCostsum[dr].append(costTemp[dr])
    driverCostsum = pd.DataFrame(driverCostsum)   
    return driverCostsum

def getCost(driverCostsum):
    solMin = []
    drMin = {'cost':[], 'driver':[], 'cluster':[]}
    for a in range(len(driverCostsum)):
        minDist = np.nanmin(driverCostsum.iloc[a])
        minDriver = getminDriver(driverCostsum, minDist, a)
        drMin['cost'].append(minDist)
        drMin['driver'].append(minDriver)
        drMin['cluster'].append(a)
    return drMin

def getdfsolTempsum(dfsolTemp1):
    solTempsum = {'cluster': [], 'totdist':[], 'ord':[], 'area':[]}
    solTempsum['cluster'] = list(dfsolTemp1['cluster'].unique())
    for a in solTempsum['cluster']:
        clCheck = dfsolTemp1[dfsolTemp1['cluster'] == a]
        solTempsum['totdist'].append(sum(clCheck['dist']))
        solTempsum['ord'].append(len(clCheck['cluster']))
        solTempsum['area'].append(list(dfsolTemp1[dfsolTemp1['cluster'] == a]['area'].unique()))
    dfsolTempsum = pd.DataFrame(solTempsum)
    return dfsolTempsum

def getdriverCost(dP, dtS, cluster):
    driverCost = {i:[] for i in dP['driver']}
    for a in range(0, len(dP['driver'])):
        fs = True
        for b in dtS['area'][cluster]:
            for c in dP['areaun'][a]:
                if b == c:
                    fs = False
        if fs:
            if dP['spec'][a] > 0:
                specCost = dP['spec'][a] + (dP['dist'][a]*max(0, dtS['totdist'][cluster] - dP['specDist'][a]))
                distCost = 0
                ordCost = 0
            else:
                specCost = 0
                distCost = dP['dist'][a]*dtS['totdist'][cluster]
                ordCost = dP['ord'][a]*dtS['ord'][cluster]
        else:
            distCost = np.nan
            ordCost = np.nan
            specCost = np.nan
        nameDriver = dP['driver'][a]
        driverCost[nameDriver] = (distCost + ordCost + specCost)
    return driverCost

def fnObj(dfsolTemp2, driverPrice):
    cost = {}
    dfsolTemp = dfsolTemp2.copy()
    dfsolTempsum = getdfsolTempsum(dfsolTemp)
    driverCostsum = getdriverCostsum(driverPrice, dfsolTempsum)
    cost = getCost(driverCostsum)
    return cost

In [23]:
"""getDist"""
def findDist(clusfindDist1, dismat):
    dist = []
    for i in range(0, len(clusfindDist1)-1):
        fromVisit = clusfindDist1[i]
        toVisit = clusfindDist1[i+1]
        d = dismat[fromVisit][int(toVisit)]
        dist.append(d)
    return dist

def findtempClus(da1):
    tempClus = {i:[] for i in da1['cluster'].unique()}
    for i in da1['cluster'].unique():
        tempClus[i].append('0')
        for a in da1.index:
            if i == da1['cluster'][a]:
                tempClus[i].append(da1['route'][a])
    return tempClus

def findclusDist(tempClus1, da1, dismat):
    clusDist = []
    for a in tempClus1:
        clusfindDist = tempClus1[a]
        apa = findDist(clusfindDist, dismat)
        clusDist.extend(apa)
    da1['dist'] = clusDist
    return da1

def getDist(da2, dismat):
    da1 = da2.copy()
    temp = findtempClus(da1)
    da1 = findclusDist(temp, da1, dismat)
    return da1

def clusterIt(dfsolTemp1, dismat, capMax, boxMax, capMin, ranRate):  
    dfsolTemp1 = dfsolTemp1.reset_index(drop = True)
    da = {'route':[i for i in dfsolTemp1['route']], 
          'lat':[i for i in dfsolTemp1['lat']],
          'lon': [i for i in dfsolTemp1['lon']], 
          'cluster':[], 
          'area':[i for i in dfsolTemp1['area']], 
          'box':[i for i in dfsolTemp1['box']], 
          'demand':[i for i in dfsolTemp1['demand']],
          'deg': [i for i in dfsolTemp1['deg']]
         }
    ba = dfsolTemp1.copy()
    numClus = 0
    capMcounter = capMax
    boxMcounter = boxMax
    for i in range(len(ba)):
        capAva = True
        boxAva = True
        
        deVisit = ba['demand'][i]
        boVisit = ba['box'][i]

        if boVisit > boxMcounter:
            boxAva = False
        if deVisit > capMcounter:
            capAva = False

        if capAva == False:
            numClus = numClus + 1
            boxMcounter = boxMax - ba['box'][i]
            capMcounter = capMax - ba['demand'][i]
            da['cluster'].append(str(numClus))
        else:
            if boxAva:
                boxMcounter = boxMcounter - ba['box'][i]
                capMcounter = capMcounter - ba['demand'][i]
                da['cluster'].append(str(numClus))
            else:
                numClus = numClus + 1
                boxMcounter = boxMax - ba['box'][i]
                capMcounter = capMax - ba['demand'][i]
                da['cluster'].append(str(numClus))

    da = pd.DataFrame(da)
    fs = getfsCluster(da, capmaxBox, capminDem, capmaxDem)
    da = da.reset_index(drop = True)
    if fs:
        da = getDist(da, dismat)
        return da, fs
    else:
        return da, fs
    
def clusterIt2(dfsolTemp1, dismat, capMax, boxMax, capMin, ranRate):  
    dfsolTemp1 = dfsolTemp1.reset_index(drop = True)
    da = {'route':[i for i in dfsolTemp1['route']], 
          'lat':[i for i in dfsolTemp1['lat']],
          'lon': [i for i in dfsolTemp1['lon']], 
          'cluster':[], 
          'area':[i for i in dfsolTemp1['area']], 
          'box':[i for i in dfsolTemp1['box']], 
          'demand':[i for i in dfsolTemp1['demand']],
          'deg': [i for i in dfsolTemp1['deg']]
         }
    ba = dfsolTemp1.copy()
    numClus = 0
    capMcounter = capMax
    boxMcounter = boxMax
    for i in range(len(ba)):
        capAva = True
        boxAva = True

        deVisit = ba['demand'][i]
        boVisit = ba['box'][i]

        if boVisit > boxMcounter:
            boxAva = False
        
        if deVisit > capMcounter:
            capAva = False
        
        if capMcounter < capMax - capMin:
            if ra.random() < ranRate:
                capAva = False
        
        if capAva == False:
            numClus = numClus + 1
            boxMcounter = boxMax - ba['box'][i]
            capMcounter = capMax - ba['demand'][i]
            da['cluster'].append(str(numClus))
        else:
            if boxAva:
                boxMcounter = boxMcounter - ba['box'][i]
                capMcounter = capMcounter - ba['demand'][i]
                da['cluster'].append(str(numClus))
            else:
                numClus = numClus + 1
                boxMcounter = boxMax - ba['box'][i]
                capMcounter = capMax - ba['demand'][i]
                da['cluster'].append(str(numClus))

    da = pd.DataFrame(da)
    fs = getfsCluster(da, capmaxBox, capminDem, capmaxDem)
    da = da.reset_index(drop = True)
    if fs:
        da = getDist(da, dismat)
        return da, fs
    else:
        return da, fs

In [68]:
"""GA parameter setting"""
cont = True
maxGen = 10000
replCa = 2
maxIter = 200
mutRate = 0.1
numChild = 2
numParents = 2
popSize = 5
ranRate = 0.1
driverPrice = {'driver':['AAA', 'BBB', 'CCC', 'DDD'], 'dist':[0, 0, 3000, 1800],
               'ord':[15000, 16000, 3000, 0], 'spec':[0,0,0,12500], 'specDist':[0,0,0,5], 
               'areaun': [[],['B'],['B', 'C'], ['A']]}

In [69]:
ra.seed(2)
numClient = 20
capmaxDem = 5
capmaxBox = 4
capminDem = 2

dismat = {
"0":[0, 13.1, 10.69, 14.27, 5.63, 2.54, 3.13, 5.81, 5.96, 3.79, 9.08, 12.96, 10.4, 2.5, 7.93, 7.46, 1.57, 4.73, 11.49, 7.39, 11.96],
"1":[13.1, 0, 3.1, 23.6, 7.61, 15.61, 10.52, 7.59, 18.79, 16.44, 4.02, 3.82, 22.76, 12.54, 13.85, 20.55, 14.31, 12.33, 1.62, 8.4, 4.77],
"2":[10.69, 3.1, 0, 22.54, 5.05, 13.22, 7.85, 4.92, 16.57, 14.27, 2.13, 2.73, 20.78, 9.78, 13.04, 18.03, 12.04, 9.34, 2.19, 5.31, 6.14],
"3":[14.27, 23.6, 22.54, 0, 18.33, 12.89, 17.05, 18.72, 10.45, 11.1, 20.43, 25.25, 7.72, 16.73, 9.77, 12.96, 12.79, 18.95, 22.13, 21.22, 19.82],
"4":[5.63, 7.61, 5.05, 18.33, 0, 8.17, 2.93, 0.46, 11.54, 9.26, 3.61, 7.4, 15.83, 4.97, 9.56, 13, 7.01, 5.28, 6.05, 3.47, 7.7],
"5":[2.54, 15.61, 13.22, 12.89, 8.17, 0, 5.56, 8.34, 3.52, 1.79, 11.61, 15.47, 8.08, 4.28, 8.48, 5, 1.51, 6.33, 14, 9.7, 14.22],
"6":[3.13, 10.52, 7.85, 17.05, 2.93, 5.56, 0, 2.95, 9.06, 6.93, 6.53, 9.94, 13.53, 2.06, 9.42, 10.2, 4.69, 3.06, 8.98, 4.28, 10.33],
"7":[5.81, 7.59, 4.92, 18.72, 0.46, 8.34, 2.95, 0, 11.75, 9.49, 3.65, 7.16, 16.08, 4.95, 10, 13.11, 7.24, 5.06, 6.07, 3.01, 7.96],
"8":[5.96, 18.79, 16.57, 10.45, 11.54, 3.52, 9.06, 11.75, 0, 2.36, 14.84, 18.91, 4.59, 7.78, 9.16, 3.19, 4.53, 9.72, 17.17, 13.22, 16.87],
"9":[3.79, 16.44, 14.27, 11.1, 9.26, 1.79, 6.93, 9.49, 2.36, 0, 12.5, 16.65, 6.61, 5.93, 7.53, 4.9, 2.26, 8.06, 14.82, 11.17, 14.56],
"10":[9.08, 4.02, 2.13, 20.43, 3.61, 11.61, 6.53, 3.65, 14.84, 12.5, 0, 4.86, 18.92, 8.57, 10.91, 16.53, 10.33, 8.61, 2.45, 5.24, 5],
"11":[12.96, 3.82, 2.73, 25.25, 7.4, 15.47, 9.94, 7.16, 18.91, 16.65, 4.86, 0, 23.23, 11.7, 15.77, 20.12, 14.4, 10.83, 4.07, 6.5, 8.2],
"12":[10.4, 22.76, 20.78, 7.72, 15.83, 8.08, 13.53, 16.08, 4.59, 6.61, 18.92, 23.23, 0, 12.35, 11.06, 5.77, 8.87, 14.3, 21.14, 17.75, 20.25],
"13":[2.5, 12.54, 9.78, 16.73, 4.97, 4.28, 2.06, 4.95, 7.78, 5.93, 8.57, 11.7, 12.35, 0, 10.17, 8.44, 3.94, 2.23, 11.01, 5.54, 12.35],
"14":[7.93, 13.85, 13.04, 9.77, 9.56, 8.48, 9.42, 10, 9.16, 7.53, 10.91, 15.77, 11.06, 10.17, 0, 12.26, 7.27, 12.22, 12.42, 12.9, 10.11],
"15":[7.46, 20.55, 18.03, 12.96, 13, 5, 10.2, 13.11, 3.19, 4.9, 16.53, 20.12, 5.77, 8.44, 12.26, 0, 6.46, 9.87, 18.95, 13.94, 19.2],
"16":[1.57, 14.31, 12.04, 12.79, 7.01, 1.51, 4.69, 7.24, 4.53, 2.26, 10.33, 14.4, 8.87, 3.94, 7.27, 6.46, 0, 6.16, 12.69, 8.96, 12.75],
"17":[4.73, 12.33, 9.34, 18.95, 5.28, 6.33, 3.06, 5.06, 9.72, 8.06, 8.61, 10.83, 14.3, 2.23, 12.22, 9.87, 6.16, 0, 10.94, 4.34, 12.99],
"18":[11.49, 1.62, 2.19, 22.13, 6.05, 14, 8.98, 6.07, 17.17, 14.82, 2.45, 4.07, 21.14, 11.01, 12.42, 18.95, 12.69, 10.94, 0, 7.22, 4.15],
"19":[7.39, 8.4, 5.31, 21.22, 3.47, 9.7, 4.28, 3.01, 13.22, 11.17, 5.24, 6.5, 17.75, 5.54, 12.9, 13.94, 8.96, 4.34, 7.22, 0, 10.18],
"20":[11.96, 4.77, 6.14, 19.82, 7.7, 14.22, 10.33, 7.96, 16.87, 14.56, 5, 8.2, 20.25, 12.35, 10.11, 19.2, 12.75, 12.99, 4.15, 10.18, 0]
}
clientDict = {
    'route':[str(i+1) for i in range(numClient)],
    'demand':[1 for i in range(numClient)],
    'box': [ra.choice([0,0,1,1,2]) for i in range(numClient)],
    'area':['A',
'A',
'C',
'A',
'B',
'B',
'A',
'C',
'A',
'A',
'A',
'C',
'B',
'A',
'B',
'A',
'B',
'A',
'B',
'A']
}

loclien = [[0, 0], 
[4.16122308585734, -12.4209312932009], 
[1.68210558183939, -10.5543723517121], 
[9.61822565915437, 10.5395631968424], 
[0.862075825114679, -5.5672323292154], 
[-0.450190424524839, 2.49614468239108], 
[-0.655342800137521, -3.06456749041989], 
[0.463712593028714, -5.79083830953606], 
[0.224381232402717, 5.95574092653961], 
[0.879563073317891, 3.69060746170296], 
[2.69266366736862, -8.67622232421967], 
[0.380244078513658, -12.9549328313349], 
[1.90372030762163, 10.2243709487002], 
[-1.99688719883004, -1.49752275032616], 
[7.87945492088182, 0.921939105842338], 
[-2.8169013493983, 6.90474841765291], 
[0.631233761890638, 1.44258590274205], 
[-3.71381051070308, -2.92713445028237], 
[3.85536917252025, -10.8289324047313], 
[-2.28206548114695, -7.02727480277083], 
[7.6630322212817, -9.18704558825405]]

dfClient = pd.DataFrame(clientDict)
dfClient= getDeg(loclien, dfClient)
dfClient = sortrouteInd(dfClient)

In [71]:
resultGASS = {
        'initialCost': [], 
        'finalCost': [],
        'runTimeSW':[],
        'runTimeGA': [],
        'totdist':[],
        'numDriver':[]
}

popitListSS = []
popatListSS = []
graphListSS = []
foundListSS = []
resultGASS = pd.DataFrame(resultGASS)
print("Algoritma dimulai")
for a in range(replCa):
    print("Replikasi: ", str(a+1), "/", str(replCa))
    result, popAwal, popAkhir, graph = gaMS(dfClient, dismat, capmaxDem, capmaxBox, capminDem, popSize, loclien, driverPrice, numParents, numClient, mutRate, numChild, ranRate)
    resultGASS = resultGASS.append(result)
    popitListSS.append(pd.DataFrame(popAwal))
    popatListSS.append(pd.DataFrame(popAkhir))
    graphListSS.append(graph)
    
for a in range(len(popatListSS)):
    temp = popatListSS[a]
    temp = temp.sort_values(by=['fitness'], ascending = False)
    print("Replikasi ", a+1)
    print("Biaya pengiriman: ", 1/temp['fitness'].iloc[0])
    print("Rincian")
    print(temp['pop'].iloc[0])

Algoritma dimulai
Replikasi:  1 / 2
Mencari solusi untuk populasi awal dengan Sweep (konstruksi)
Solusi  1 ditemukan
Solusi  2 ditemukan
Solusi  3 ditemukan
Solusi  4 ditemukan
Solusi  5 ditemukan
Memperbaiki solusi dengan algoritma genetika
Biaya pengiriman:  237257.99999999997
Improved
Biaya pengiriman:  234821.99999999997
Replikasi:  2 / 2
Mencari solusi untuk populasi awal dengan Sweep (konstruksi)
Solusi  1 ditemukan
Solusi  2 ditemukan
Solusi  3 ditemukan
Solusi  4 ditemukan
Solusi  5 ditemukan
Memperbaiki solusi dengan algoritma genetika
Biaya pengiriman:  251386.0
Improved
Biaya pengiriman:  247588.00000000003
Replikasi  1
Biaya pengiriman:  234821.99999999997
Rincian
   route       lat        lon area  box  demand         deg   dist cluster  \
0      4  0.862076  -5.567232    A    1       1  351.197761   5.63       0   
1      7  0.463713  -5.790838    A    1       1  355.421699   0.46       0   
2     10  2.692664  -8.676222    A    2       1  342.758282   3.65       0   
3  